一. 個別模型訓練 M1: LR. M2: DT, M3:SVM, M4:KNN, M5:RF 二. 集成學習模型訓練 : Bagging 以上皆須附上confusion matrix, Accuracy, Precision, Recall, F1. AUC 請繳交程式及EXCEL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir("/content/drive/MyDrive/大三/下學期/機器學習應用/Datasets/ Bank Marketing")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Feature Engineering import
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.preprocessing import LabelEncoder ,OrdinalEncoder
from imblearn.over_sampling import SMOTE

# Model import
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

# Evaluate import
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_auc_score ,roc_curve ,classification_report, confusion_matrix ,precision_score, recall_score, f1_score

# sns.set(style="whitegrid")
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.max_columns', None)
pd.set_option("display.precision", 2)
plt.rcParams['axes.unicode_minus'] = False # 正常顯示負號

# EDA
讀取資料，做資料探索

In [ ]:
df = pd.read_csv("./bank-full.csv" ,sep =';')

# df # 45211 rows × 17 columns

In [ ]:
df.info()

In [ ]:
import missingno as msno

msno.bar(df)

No Missing Value

In [ ]:
# Numerical statistics
df.describe()

In [ ]:
# Object statistics
df.describe(include=['object'])

In [ ]:
numerical_columns = df.select_dtypes(include='number').columns

plt.figure(figsize=(14, 8))
for i, column in enumerate(numerical_columns, 1):
    plt.subplot(2, 4, i)
    sns.histplot(df[column], bins=20, kde=True)
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 25))
df.plot(kind="box", subplots=True, layout=(3, 3), figsize=(15, 25))
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.subplot(1 ,2 ,1)
sns.countplot(data =df ,x ='y')
plt.xticks(rotation=45)
plt.title('y Count')

plt.subplot(1 ,2 ,2)
profile_counts = df['y'].value_counts()
plt.pie(profile_counts, labels = profile_counts.index, autopct='%1.1f%%')
plt.title('y Percentage')
plt.tight_layout()
plt.show()

print(df['y'].value_counts()) # No:39922(88.3%) ,Yes:5289(11.7%)

得圖表和輸出內容得知,No:39922(88.3%) ,Yes:5289(11.7%)

# Processing Data

刪除特定欄位,做異常值處理

In [ ]:
delete_columns = ['poutcome']
df = df.drop(delete_columns, axis=1)

df = df.drop(4)

In [ ]:
numerical_columns = df.select_dtypes(include='number').columns

for column in numerical_columns:
  means = df[column].mean()
  stds = df[column].std()

  lower_bound =  means - 3 * stds
  upper_bound =  means + 3 * stds

  df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [ ]:
# Numerical statistics
df.describe()

In [ ]:
df # 45211 rows × 17 columns ---> 40339 rows × 16 columns

# Feature Engineering
將特徵和標籤做特徵工程轉換

In [ ]:
# 標籤y做LabelEncoder
lb_encoder = LabelEncoder()
df['y'] = lb_encoder.fit_transform(df['y'])
# Original values for column 'y': ['no' 'yes']
# Encoded values for column 'y': [0 1]

# 類別特徵欄位做OrdinalEncoder
cat_columns = df.select_dtypes(include='object').columns
or_encoder = OrdinalEncoder()
df[cat_columns] = or_encoder.fit_transform(df[cat_columns])

# Train Test Split
資料分割，將資料集拆分為訓練集和測試集

In [ ]:
X = df.drop(columns=['y'])
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', X_train.shape)
print('y_test:', X_test.shape)
print('='*30)
# 查看全部資料的類別比例
print(pd.Series(y).value_counts(normalize=True))
# 查看訓練集標籤的分佈
print(pd.Series(y_train).value_counts(normalize=True))
# 查看測試集的標籤分佈
print(pd.Series(y_test).value_counts(normalize=True))

In [ ]:
from collections import Counter

print(f"Training target statistics: {Counter(y_train)}")
print(f"Testing target statistics: {Counter(y_test)}")

Training target statistics: Counter({0: 25392, 1: 2845})


Testing target statistics: Counter({0: 10881, 1: 1221})

# SMOTE
SMOTE處理資料不平衡

In [ ]:
smote = SMOTE(random_state=42)
X_resample ,y_resample = smote.fit_resample(X_train, y_train)

In [ ]:
resampled_data = pd.concat([X_resample, pd.Series(y_resample, name='y')], axis=1)

resampled_data # 50784 rows × 16 columns

In [ ]:
plt.figure(figsize=(10, 6))
plt.subplot(1 ,2 ,1)
sns.countplot(data =resampled_data ,x ='y')
plt.xticks(rotation=45)
plt.title('y Count')

plt.subplot(1 ,2 ,2)
profile_counts = resampled_data['y'].value_counts()
plt.pie(profile_counts, labels = profile_counts.index, autopct='%1.1f%%')
plt.title('y Percentage')
plt.tight_layout()
plt.show()

print(resampled_data['y'].value_counts()) # No:25392(50%) ,Yes:25392(50%)

In [ ]:
X_resample = resampled_data.drop('y' ,axis =1)
y_resample = resampled_data['y']

X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, test_size=0.3, random_state=42)

print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', X_train.shape)
print('y_test:', X_test.shape)
print('='*30)
# 查看訓練集標籤的分佈
print(pd.Series(y_train).value_counts(normalize=True))
# 查看測試集的標籤分佈
print(pd.Series(y_test).value_counts(normalize=True))

In [ ]:
print(f"Training target statistics: {Counter(y_train)}")
print(f"Testing target statistics: {Counter(y_test)}")

Training target statistics: Counter({1: 17781, 0: 17767})

Testing target statistics: Counter({0: 7625, 1: 7611})

# Feature_Importances

找出重要特徵

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100 ,random_state=42)

rf_model.fit(X_train, y_train)

importances = rf_model.feature_importances_
feature_scores = pd.Series(rf_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
feature_scores

In [ ]:
plt.figure(figsize=(10, 6))
feature_scores.plot(kind='bar', color='skyblue')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importance')
plt.xticks(rotation=60)
plt.tight_layout()
plt.show()

圖表得知特徵重要性前七高依序為

1.   duration : 0.29
2.   housing : 0.14
3.   contact : 0.09
4.   education : 0.08
5.   marital : 0.07
6.   month : 0.075
7.   job : 0.06

以前七高的特徵來做模型的輸入特徵


In [ ]:
# SMOYE_before
selected_features = ['duration' ,'housing' ,'age' ,'month' ,'day' ,'pdays' ,'job']
X = df[selected_features]
y = df['y']

# SMOTE_after
# selected_features = ['duration' ,'housing' ,'contact' ,'education' ,'marital' ,'month' ,'job']
# X = resampled_data[selected_features]
# y = resampled_data['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', X_train.shape)
print('y_test:', X_test.shape)
print('='*30)
# 查看訓練集標籤的分佈
print(pd.Series(y_train).value_counts(normalize=True))
# 查看測試集的標籤分佈
print(pd.Series(y_test).value_counts(normalize=True))

# Model

In [ ]:
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

svm_model = SVC(random_state=42)
svm_model.fit(X_train ,y_train)

knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate

In [ ]:
def evaluate_model_performance(model, X_train, X_test, y_train, y_test):
    # 訓練集上的模型評分
    y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_conf_matrix = confusion_matrix(y_train, y_train_pred)
    train_classification_report = classification_report(y_train, y_train_pred)

    print("Training Set Evaluation:")
    print("Accuracy:", train_accuracy)
    print("Confusion Matrix:")
    print(train_conf_matrix)
    print("Classification Report:")
    print(train_classification_report)
    print("\n")

    # 測試集上的模型評分
    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_conf_matrix = confusion_matrix(y_test, y_test_pred)
    test_classification_report = classification_report(y_test, y_test_pred)
    test_auc = roc_auc_score(y_test, y_test_pred)

    print("Testing Set Evaluation:")
    print("Accuracy:", test_accuracy)
    print("Confusion Matrix:")
    print(test_conf_matrix)
    print("Classification Report:")
    print(test_classification_report)
    print("AUC:")
    print(test_auc)
evaluate_model_performance(lr_model, X_train, X_test, y_train, y_test)

In [ ]:
evaluate_model_performance(dt_model, X_train, X_test, y_train, y_test)

In [ ]:
evaluate_model_performance(svm_model, X_train, X_test, y_train, y_test)

In [ ]:
evaluate_model_performance(knn_model, X_train, X_test, y_train, y_test)

In [ ]:
evaluate_model_performance(rf_model, X_train, X_test, y_train, y_test)

# PipeLine

In [ ]:
from sklearn.pipeline import Pipeline
# Pipeline
lr_pipeline = Pipeline(steps=[
    ('lr', LogisticRegression(random_state=42))])

dt_pipeline = Pipeline(steps=[
    ('dt', DecisionTreeClassifier(random_state=42))])

svm_pipeline = Pipeline(steps=[
    ('svm', SVC(random_state=42))])

knn_pipeline = Pipeline(steps=[
    ('knn', KNeighborsClassifier())])

rf_pipeline = Pipeline(steps=[
    ('rf', RandomForestClassifier(random_state=42))])

In [ ]:
models = {
    'Logistic Regression': lr_pipeline,
    'Decision Tree': dt_pipeline,
    'SVM': svm_pipeline,
    'KNN': knn_pipeline,
    'Random Forest': rf_pipeline
}

test_scores = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    test_accuracy = accuracy_score(y_test, y_pred)
    test_conf_matrix = confusion_matrix(y_test, y_pred)
    test_class_report = classification_report(y_test, y_pred)
    test_auc = roc_auc_score(y_test, y_pred)

    test_scores[name] = {
        'Accuracy': test_accuracy,
        'Confusion Matrix': test_conf_matrix,
        'Classification Report': test_class_report,
        'AUC': test_auc
    }

In [ ]:
for model_name, scores in test_scores.items():
    print(f"----- Model: {model_name} -----")
    print(f"Accuracy: {scores['Accuracy']}")
    print(f"Confusion Matrix:\n{scores['Confusion Matrix']}")
    print(f"Classification Report:\n{scores['Classification Report']}")
    print(f"AUC: {scores['AUC']}\n")

# Bagging

In [ ]:
# 決策樹當基模型
model = dt_model

bagging_model = BaggingClassifier(estimator=model, n_estimators=10 ,random_state=42)
bagging_model.fit(X_train, y_train)

evaluate_model_performance(bagging_model, X_train, X_test, y_train, y_test)

# ROC AUC

In [ ]:
plt.figure(figsize=(10, 8))

lr_y_score = lr_model.decision_function(X_test)
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr_y_score)
roc_auc_lr = roc_auc_score(y_test, lr_y_score)
plt.plot(fpr_lr, tpr_lr, color='blue', lw=2, label=f'Logistic Regression (AUC = {roc_auc_lr:.2f})')

dt_y_score = dt_model.predict_proba(X_test)[:, 1]
fpr_dt, tpr_dt, _ = roc_curve(y_test, dt_y_score)
roc_auc_dt = roc_auc_score(y_test, dt_y_score)
plt.plot(fpr_dt, tpr_dt, color='green', lw=2, label=f'Decision Tree (AUC = {roc_auc_dt:.2f})')

svm_y_score = svm_model.decision_function(X_test)
fpr_svm, tpr_svm, _ = roc_curve(y_test, svm_y_score)
roc_auc_svm = roc_auc_score(y_test, svm_y_score)
plt.plot(fpr_svm, tpr_svm, color='red', lw=2, label=f'SVM (AUC = {roc_auc_svm:.2f})')

knn_y_score = knn_model.predict_proba(X_test)[:, 1]
fpr_knn, tpr_knn, _ = roc_curve(y_test, knn_y_score)
roc_auc_knn = roc_auc_score(y_test, knn_y_score)
plt.plot(fpr_knn, tpr_knn, color='orange', lw=2, label=f'KNN (AUC = {roc_auc_knn:.2f})')

rf_y_score = rf_model.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_y_score)
roc_auc_rf = roc_auc_score(y_test, rf_y_score)
plt.plot(fpr_rf, tpr_rf, color='purple', lw=2, label=f'Random Forest (AUC = {roc_auc_rf:.2f})')

bagging_y_score = bagging_model.predict_proba(X_test)[:, 1]
fpr_bagging, tpr_bagging, _ = roc_curve(y_test, bagging_y_score)
roc_auc_bagging = roc_auc_score(y_test, bagging_y_score)
plt.plot(fpr_bagging, tpr_bagging, color='black', lw=2, label=f'Bagging (AUC = {roc_auc_bagging:.2f})')

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()